In [259]:
%pylab inline
import pandas as pd
import seaborn
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics.pairwise import manhattan_distances

Populating the interactive namespace from numpy and matplotlib


In [260]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')
import cv2

In [261]:
# from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

## TRAINING TIME!

In [262]:
# new_df.to_json('/Users/Home/Desktop/CMPT 459/MI2/train_fe.json')
train_df = pd.read_json(r'C:\Users\Giovanni PC\cmpt459\data\final_train_df.json')

In [263]:
train_df

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,...,floor,high,kitchen,room,mall_distance,created_hour,created_month,created_day,num_photos,description_len
0,1.0,1,8579a0b0d54db803821a35a4a615e97a,1466056527000,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,...,0.110607,0.000000,0.098530,0.000000,0.04995,5,6,16,12,77
1,1.0,2,b8e75fc949a6cd8225b455648a951712,1464759873000,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,...,0.000000,0.109239,0.000000,0.000000,0.01463,5,6,1,6,131
2,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,1465917599000,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,...,0.000000,0.000000,0.063995,0.086122,0.01813,15,6,14,6,119
3,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,1466754864000,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,...,0.085758,0.000000,0.076394,0.205618,0.06505,7,6,24,5,95
4,1.0,0,bfb9405149bfff42a92980b594c28234,1467085823000,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,...,0.000000,0.000000,0.000000,0.000000,0.01993,3,6,28,4,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48324,1.0,3,92bbbf38baadfde0576fc496bd41749c,1459828713000,There is 700 square feet of recently renovated...,W 171 Street,"[Elevator, Dishwasher, Hardwood Floors]",40.8433,6824800,-73.9396,...,0.071316,0.000000,0.063529,0.000000,0.03042,3,4,5,5,131
48325,1.0,2,5565db9b7cba3603834c4aa6f2950960,1459563931000,"2 bedroom apartment with updated kitchen, rece...",Broadway,"[Common Outdoor Space, Cats Allowed, Dogs Allo...",40.8198,6813268,-73.9578,...,0.000000,0.000000,0.104327,0.000000,0.02884,2,4,2,5,70
48326,1.0,1,67997a128056ee1ed7d046bbb856e3c7,1461649323000,No Brokers Fee * Never Lived 1 Bedroom 1 Bathr...,210 Brighton 15th St,"[Dining Room, Elevator, Pre-War, Laundry in Bu...",40.5765,6927093,-73.9554,...,0.000000,0.000000,0.000000,0.000000,0.09354,5,4,26,3,81
48327,1.0,2,3c0574a740154806c18bdf1fddd3d966,1461034053000,Wonderful Bright Chelsea 2 Bedroom apartment o...,West 21st Street,"[Pre-War, Laundry in Unit, Dishwasher, No Fee,...",40.7448,6892816,-74.0017,...,0.080766,0.000000,0.071947,0.000000,0.00671,2,4,19,5,118


In [264]:
train_df.describe()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48329 entries, 0 to 48328
Data columns (total 36 columns):
bathrooms            48329 non-null float64
bedrooms             48329 non-null int64
building_id          48329 non-null object
created              48329 non-null int64
description          48329 non-null object
display_address      48329 non-null object
features             48329 non-null object
latitude             48329 non-null float64
listing_id           48329 non-null int64
longitude            48329 non-null float64
manager_id           48329 non-null object
photos               48329 non-null object
price                48329 non-null int64
street_address       48329 non-null object
interest_level       48329 non-null object
dt_distance          48329 non-null object
pet_policy           48329 non-null int64
listing_type         48329 non-null int64
unit_features        48329 non-null int64
building_features    48329 non-null int64
apartment            48329 non-null 

In [265]:
# New section
downtown = [40.7209, -74.0007]

def man_distances(row):
    dist = manhattan_distances([[row.latitude, row.longitude]], [downtown])
    return dist[0][0]


train_df['dt_distance'] = train_df.apply(lambda row: man_distances(row), axis=1)
train_df.dt_distance.head()

0    0.0569
1    0.0589
2    0.0748
3    0.0646
4    0.0494
Name: dt_distance, dtype: float64

In [266]:
selected_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
                  "dt_distance", "pet_policy", "listing_type", "unit_features", "building_features", "apartment",
                  "appliance", "bathroom", "bedroom", "building", "contact", "floor", "high", "kitchen", "room", 
                  "mall_distance", "created_hour", "created_month", "created_day", "num_photos", "description_len"]

In [267]:
X = train_df[selected_feats]
y = train_df["interest_level"]

In [268]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [269]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

In [270]:
X

array([[0.1       , 0.125     , 0.27075252, ..., 0.5       , 0.17647059,
        0.11411411],
       [0.1       , 0.25      , 0.34930178, ..., 0.        , 0.08823529,
        0.1951952 ],
       [0.1       , 0.25      , 0.36132661, ..., 0.43333333, 0.08823529,
        0.17717718],
       ...,
       [0.1       , 0.125     , 0.01027929, ..., 0.83333333, 0.04411765,
        0.12012012],
       [0.1       , 0.25      , 0.33669511, ..., 0.6       , 0.07352941,
        0.17567568],
       [0.1       , 0.375     , 0.36501164, ..., 0.63333333, 0.11764706,
        0.08558559]])

In [271]:
#Train for decision Tree
test_scores = []
train_scores = []
log_scores = []
log_scores_train = []

min_sample_arr = [1, 50, 100, 200, 500] 

i = 0
cv = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in cv.split(X):
    dectree  = DecisionTreeClassifier(splitter='random', max_features='sqrt', random_state=0, min_samples_leaf=min_sample_arr[i], max_depth=10)
    i += 1
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y.iloc[train_index], y.iloc[test_index]
    dectree.fit(X_train, y_train)
    test_scores.append(dectree.score(X_test, y_test))
    train_scores.append(dectree.score(X_train, y_train))
    y_test_pred = dectree.predict_proba(X_test)
    y_train_pred = dectree.predict_proba(X_train)
    log_scores.append(log_loss(y_test, y_test_pred))
    log_scores_train.append(log_loss(y_train, y_train_pred))
    
    #when the min sample change, there is pverfitting for small values,
    #as the calues grow, the training set does progressively worse in comparison to the test 
    # use 50 as the number
    

In [272]:
log_scores


[0.8974916722462367,
 0.7496315236517167,
 0.7307054020817676,
 0.763572054013412,
 0.7596309960637618]

In [273]:
log_scores_train

[0.7240336477013386,
 0.7263551512257378,
 0.7361892272645845,
 0.7459331861570265,
 0.763558545341905]

In [274]:
y_train_pred


array([[0.08130224, 0.66869042, 0.25000733],
       [0.08130224, 0.66869042, 0.25000733],
       [0.08130224, 0.66869042, 0.25000733],
       ...,
       [0.08130224, 0.66869042, 0.25000733],
       [0.08130224, 0.66869042, 0.25000733],
       [0.08130224, 0.66869042, 0.25000733]])

In [275]:
labels2idx = {label: i for i, label in enumerate(dectree.classes_)}
labels2idx

{'high': 0, 'low': 1, 'medium': 2}

In [276]:
# new_df.to_json('/Users/Home/Desktop/CMPT 459/MI2/train_fe.json')
test_df = pd.read_json(r'C:\Users\Giovanni PC\cmpt459\data\final_test.json')

In [277]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

In [278]:
dectree  = DecisionTreeClassifier(splitter='random', max_features='sqrt', random_state=0, min_samples_leaf=200, max_depth=10)
dectree.fit(X_train, y_train)
y_val_pred = dectree.predict_proba(X_val)
log_loss(y_val, y_val_pred)

0.746840501859756

In [279]:
test_X = test_df[selected_feats]

In [280]:
test_X = scaler.fit_transform(test_X)

In [281]:
test_y = dectree.predict_proba(test_X)

In [282]:
labels2idx = {label: i for i, label in enumerate(dectree.classes_)}
labels2idx

{'high': 0, 'low': 1, 'medium': 2}

In [283]:
sub = pd.DataFrame()
sub["listing_id"] = test_df["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = test_y[:, labels2idx[label]]
sub.to_csv(r"C:\Users\Giovanni PC\cmpt459\data\submission_dt.csv", index=False)
